In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import torch
# from fake_claims_generator import generate, grab_random_prompts

In [ ]:
!pip3 install git+https://github.com/huggingface/transformers@v3.1.0 # there are errors in the later versions

  Cloning https://github.com/huggingface/transformers (to revision v3.1.0) to /tmp/pip-req-build-txhi1i6_
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-txhi1i6_
  Running command git checkout -q 4f0a753715e2a4b521db2e91200d947114e0c90c
     |████████████████████████████████| 3.0MB 2.8MB/s 
     |████████████████████████████████| 1.1MB 17.2MB/s 
     |████████████████████████████████| 890kB 35.3MB/s 
  Created wheel for transformers: filename=transformers-3.1.0-cp36-none-any.whl size=884019 sha256=d782f020d88bbc8f63178257046b28acb88ae41d1b58a4d3614419c9527acf8a
  Stored in directory: /tmp/pip-ephem-wheel-cache-1yn05h3x/wheels/97/58/f3/fab46f0e4c13b26fe3bb745b20a17792a12b324163b2cfe393
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=2dbf0c425e930600fd711e6ae1f6fcb1585ab73ccf5ff6d483fdad1881520c91
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
%cd drive/My\ Drive/School\ Work/Fourth\ Year/Capstone\ Everything

/content/drive/My Drive/School Work/Fourth Year/Capstone Everything


In [ ]:
def grab_random_prompts(data, numprompts, first_words=5, column='claims'):
    '''
    Inputs:
    numprompts int: Number of prompts we would want
    first_words int: How many first words would we want
    '''
    num_data = len(data)
    prompts = []
    while len(prompts) < numprompts:
        row = np.random.randint(0, num_data-1)
        try:
            words = data[column][row].split()
            firsts = " ".join(words[:first_words])
            prompts.append(firsts)
        except:
            #print("No claims")
            pass
    
#     for indx in rows:
#         words = data[column][indx].split()
#         firsts = " ".join(words[:first_words])
#         prompts.append(firsts)
    return prompts

OUTPUT_DIR = "./Models/output_abstract" # the model for abstract is stored here
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

tokenizer = GPT2Tokenizer.from_pretrained(OUTPUT_DIR)
model = GPT2LMHeadModel.from_pretrained(OUTPUT_DIR)
model = model.to(device)
def generate(input_str, length=250, n=5):
    cur_ids = torch.tensor(tokenizer.encode(input_str)).unsqueeze(0).long().to(device)
    model.eval()
    with torch.no_grad():
        for i in range(length):
            outputs = model(cur_ids[:, -1024:], labels=cur_ids[:, -1024:])
            loss, logits = outputs[:2]
            softmax_logits = torch.softmax(logits[0,-1], dim=0)
            next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n)
            cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim=1)
        output_list = list(cur_ids.squeeze().to('cpu').numpy())
        output_text = tokenizer.decode(output_list)
        return output_text

def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

In [ ]:
device

'cuda'

## Load Real Data and Label Them


In [ ]:
data = pd.read_csv("./Models/Data/H04L.csv")
real_data = pd.DataFrame({"Abstract": data["abstract"]})
real_data["Label"] = 0
real_data = real_data[real_data['Abstract'].notnull()]

In [ ]:
real_data

,Abstract,Label
0,An organic light emitting diode display includ...,0
1,Provided is an electric element cooling module...,0
2,An organic light emitting diode display includ...,0
3,Provided is an electric element cooling module...,0
4,An organic light emitting diode display includ...,0
...,...,...
8942,A method including patterning a continuous fin...,0
8943,A method including patterning a continuous fin...,0
8944,A method including patterning a continuous fin...,0
8945,A method including patterning a continuous fin...,0


## Generate Fake Data and Label Them

In [ ]:
# How many fake claims do you want to generate
# num_fake_claims = len(real_data)
# prompts = grab_random_prompts(data, num_fake_claims, first_words=5)
# fake_claims = []
# for i, prompt in enumerate(prompts):
#     print(f"Generating Prompt {i+1} ... Prompt: {prompt}")
#     generated_text = generate(prompt, n=8)
#     fake_claims.append(generated_text)
  

In [ ]:
# write in chunks, and write to csv
import os.path
fake_data_file = "Models/Data/real_fake_patent_abstracts/fake_patent_abstracts.csv"
prompt_file = "Models/Data/real_fake_patent_abstracts/prompts.csv"
num_fake_claims = len(real_data)
if os.path.exists(prompt_file):
  prompts = pd.read_csv(prompt_file)
  prompts = prompts['Prompts'].tolist()
else:
  prompts = grab_random_prompts(data, num_fake_claims, first_words=5, column='abstract')
  # save prompts
  prompt_df = pd.DataFrame({"Prompts": prompts})
  prompt_df.to_csv(prompt_file)
fake_abstracts = []
last_interupted = 5300
for i, prompt in enumerate(prompts[last_interupted:]): # modify last_interupted if we got disconnected
    if i % 100 == 0 and i != 0: # for every 100 claims
      df = pd.DataFrame({"Abstract": fake_abstracts, "Label": 1})
      if os.path.exists(fake_data_file): # this means this csv is already created
        df.to_csv(fake_data_file, mode='a', header=False)
      else:
        df.to_csv(fake_data_file)
      fake_abstracts = []
    print(f"Generating Prompt {i+1} ... Prompt: {prompt}")
    generated_text = generate(prompt, n=8)
    fake_abstracts.append(generated_text)
# last batch (that doesn't go over 100)
df = pd.DataFrame({"Claims": fake_abstracts, "Label": 1})
df.to_csv(fake_data_file, mode='a', header=False)

Generating Prompt 1 ... Prompt: An organic light emitting diode
Generating Prompt 2 ... Prompt: The present specification relates to
Generating Prompt 3 ... Prompt: A semiconductor device includes an
Generating Prompt 4 ... Prompt: A method of manufacturing a
Generating Prompt 5 ... Prompt: Disclosed are a method for
Generating Prompt 6 ... Prompt: An application specific integrated circuit
Generating Prompt 7 ... Prompt: A method is presented for
Generating Prompt 8 ... Prompt: A composition comprising a first
Generating Prompt 9 ... Prompt: A light-emitting element that includes
Generating Prompt 10 ... Prompt: Provided are a compound represented
Generating Prompt 11 ... Prompt: One aspect of the present
Generating Prompt 12 ... Prompt: According to one example, a
Generating Prompt 13 ... Prompt: A device including oxide spacer
Generating Prompt 14 ... Prompt: An imaging element includes: an
Generating Prompt 15 ... Prompt: A TFT substrate includes a
Generating Prompt 16 ... Prompt: 

In [ ]:
fake_data = pd.read_csv("./Models/Data/real_fake_patent_abstracts/fake_patent_abstracts.csv")
fake_data = fake_data.loc[:, ~fake_data.columns.str.contains('^Unnamed')]
fake_data

,Abstract,Label
0,The invention relates to an organic light emit...,1
1,Proposed is a layer structure comprising a fir...,1
2,A light emitting diode is disposed on a base s...,1
3,The present disclosure provides a method and a...,1
4,Disclosed is an infrared light emitting diode ...,1
...,...,...
8942,Disclosed herein is an organic light emitting ...,1
8943,The present invention relates to a composition...,1
8944,A display device is described. In one embodime...,1
8945,An organic EL display device with a subst...,1


In [ ]:
len(fake_data)

8947

In [ ]:
fake_prompts = pd.read_csv("Models/Data/real_fake_patent_abstracts/prompts.csv")

In [ ]:
fake_data['Abstract'][0]

'The invention relates to an organic light emitting device      and an organic emissive layer, especially for use in organic      light-emitting diode applications. In an example, the organic light     emitting device includes a cathode, an EL layer, and an EL layer. The EL     layer includes a host material and an emissive layer. The host      material is an emissive material having a structure in which a plurality of      holes have an amorphous or transparent state, with or without exposed surface      regions, and the plurality of holes includes a plurality of organic      light-emitting pixels, each pixel having an area of a predetermined      area. The plurality of holes includes a light-emitting layer that includes a      reflective material. The light-emitting layer has a first partial region,      a second partial region, and a third partial region. The      light-emitting layer has a polarization of one of the first partial      region and the second partial region. The polar

In [ ]:
fake_prompts['Prompts'][0]

'The invention relates to an'

In [ ]:
frames = [real_data, fake_data]
full_data = pd.concat(frames)


full_data = full_data.sample(frac=1).reset_index(drop=True)


In [ ]:
full_data


,Abstract,Label
0,A method of manufacturing a substrate inc...,1
1,A semiconductor device includes a substrate ha...,0
2,Discussed is a method of manufacturing a solar...,0
3,A Doherty amplifier includes a metal baseplate...,0
4,A method is provided for preparing a semicondu...,0
...,...,...
17889,A group III nitride semiconductor light-emitti...,0
17890,An organic light-emitting display apparatus in...,0
17891,A semiconductor light emitting device having a...,1
17892,A semiconductor device includes a first electr...,1


In [ ]:
full_data.to_csv("./Models/Data/real_fake_patent_abstracts/real_fake_abstracts.csv")